# *Construcción de dashboards interactivos con Dash*, usando datos financieros y de ventas descargables de portales públicos. 

> **Fuentes y datasets** recomendados (puedes bajar datos desde estas fuentes): Yahoo Finance (descarga histórica CSV/manual o vía `yfinance`), Alpha Vantage (API), FRED (macro series), y conjuntos de retail en Kaggle. ([macroption.com][1], [alphavantage.co][2], [fredhelp.stlouisfed.org][3], [kaggle.com][4])

---


# 1. TEORÍA

## 1.1 ¿Qué es un dashboard interactivo?

Un **dashboard** es una interfaz visual pensada para comunicar indicadores (KPIs) y facilitar la toma de decisiones. Un **dashboard interactivo** permite explorar, filtrar y profundizar en la información en tiempo real o con datos históricos.

**Diferencia con un reporte estático:**

* Reporte: un snapshot (PDF/Excel).
* Dashboard: interacción (filtrado, zoom temporal, drill-down), enlaces entre gráficas (cross-filtering), actualizaciones programadas.

Tipos (y su enfoque educativo):

* **Operativo:** latencia baja, muestra estados y alertas (ej. ventas del día, errores de pipeline).
* **Táctico:** seguimiento por período (ej. KPIs semanales, campañas).
* **Estratégico:** visión agregada para directivos (ej. CAGR, tendencias a 5 años).

---



## 1.2 Arquitectura general de un dashboard con Dash

Capas:

1. **Fuente de datos**: CSV, API (Yahoo, Alpha Vantage), bases transaccionales. ([macroption.com][1], [alphavantage.co][2])
2. **Ingesta / ETL**: limpieza, resampleo, agregación, cálculo de KPIs.
3. **Capa analítica**: métricas, indicadores financieros, cálculos de series temporales.
4. **Capa de visualización**: Dash (servidor Flask + React + Plotly).
5. **Distribución/operación**: despliegue (Docker / Gunicorn / nginx / cloud), caching y monitorización.

En Dash, los elementos clave son:



* `app = Dash(__name__)` (inicialización).
* `layout` (estructura con `html.Div`, `dcc.Graph`, `dcc.Dropdown`, etc.).
* `callbacks` (Input → lógica → Output).

---


## 1.3 Buenas prácticas de diseño UX/visual

* **KISS**: prioriza la claridad.
* **Jerarquía visual**: KPIs arriba (valores grandes) → gráficas de soporte.
* **Evita saturación**: máximo 3-5 visualizaciones por pantalla.
* **Accesibilidad**: contraste, etiquetas legibles, tooltips.
* **Colores**: paleta consistente; cuidado con daltonismo (usa paletas amigables).
* **Interactividad**: controles claros (dropdowns, datepickers), indicadores de carga (spinner).

---


## 1.4 Modelado y preparación de datos (financiero/ventas)

**Time series**:

* Indexar por `Fecha` (pd.to\_datetime).
* Resampleo: `df.resample('W').sum()` o `df.resample('M').sum()` para ver tendencias.
* Missing data: imputar con forward-fill o interpolación si es diario; si faltan largos periodos, marcar como NaN y avisar.
* Outliers: winsorize o truncar valores extremos si afectan la escalabilidad de la visualización.


---

Instalar
```
pip install pandas numpy scipy yfinance dash plotly
```

---

### Enunciado de ejercicio

Una empresa tecnológica desea analizar el comportamiento de sus **ventas mensuales y semanales** a partir de datos históricos de mercado que simulan su desempeño financiero. Para ello, se tomará como ejemplo el precio de cierre de la acción de **Apple (AAPL)** en el período comprendido entre el **1 de enero de 2022 y el 31 de diciembre de 2023**, descargado directamente desde **Yahoo Finance** mediante la librería `yfinance`.

El objetivo del análisis es transformar estos datos financieros en un escenario empresarial de ventas y realizar sobre ellos distintos procesos de **limpieza, transformación, análisis estadístico y detección de anomalías**, como si se tratara de la información real de una compañía.

El programa debe:

1. **Descargar los datos históricos** de Apple (AAPL) y adaptar los nombres de columnas para simular un conjunto de datos de ventas, tomando la fecha como `Fecha` y el precio de cierre como `Ventas`.
2. **Explorar la información inicial**, mostrando los primeros registros, la cantidad total de datos y el rango temporal disponible.
3. **Reestructurar los datos en series temporales**, indexando por fecha y realizando distintos tipos de resampleo:

   * **Mensual**: sumatoria de ventas por cada mes.
   * **Semanal**: sumatoria de ventas por cada semana.
4. **Calcular el crecimiento interanual (YoY)** de las ventas en porcentaje, considerando períodos de 12 meses.
5. **Detectar y controlar outliers** en las ventas utilizando la técnica de winsorización, comparando los valores mínimos y máximos originales frente a los ajustados.
6. **Verificar la existencia de datos faltantes (NaN)** e implementar, si es necesario, un método de imputación mediante **forward-fill** (propagación hacia adelante del último valor válido).
7. **Generar un resumen estadístico** descriptivo de las ventas mensuales, que incluya media, desviación estándar, percentiles y valores extremos.

Este ejercicio permite aplicar en un caso práctico el uso combinado de **librerías de análisis de datos (`pandas`, `numpy`)**, descarga de información financiera (`yfinance`), **estadística básica**, **detección de anomalías** y **técnicas de preprocesamiento de datos reales**.

---

In [ ]:
# Importamos pandas para manipular datos en estructuras tipo DataFrame (operaciones tabulares)
import pandas as pd
# Importamos numpy para operaciones numéricas y arrays (útil para cálculos estadísticos)
import numpy as np
# yfinance se usa para obtener datos de mercado (Yahoo Finance) sin necesidad de gestión manual de CSV
import yfinance as yf
# Importamos utilidades de fecha/hora (aquí no se usan explícitamente luego, pero suelen ser útiles)
from datetime import datetime, timedelta

# Mensaje informativo por consola indicando que se inicia la descarga de datos
print("Descargando datos de Apple (AAPL) desde Yahoo Finance...")
# Creamos un objeto que representa el ticker "AAPL" y expone métodos para obtener datos e info
ticker = yf.Ticker("AAPL")
# Descargamos el histórico de precios entre las fechas indicadas; devuelve un DataFrame con índice datetime
df = ticker.history(start="2022-01-01", end="2023-12-31")

# --- Transformaciones iniciales para adaptar los nombres a un ejemplo de "ventas" ---

# Convertimos el índice (DatetimeIndex) en una columna llamada 'Date' — reset_index crea esa columna
df = df.reset_index()
# Renombramos columnas: 'Date' → 'Fecha' y 'Close' → 'Ventas' para que el ejemplo use términos de ventas
df = df.rename(columns={'Date': 'Fecha', 'Close': 'Ventas'})
# Seleccionamos únicamente las columnas 'Fecha' y 'Ventas', descartando Open/High/Low/Volume
df = df[['Fecha', 'Ventas']]  # Solo fecha y precio de cierre como "ventas"

# Mostramos en consola un encabezado indicando que a continuación se verán los datos descargados
print("=== DATOS REALES DE APPLE (AAPL) ===")
# Mostramos las primeras 5 filas del DataFrame para inspección rápida
print(df.head())
# Imprimimos el número total de registros (filas) del DataFrame
print(f"Total de registros: {len(df)}")
# Imprimimos la fecha mínima y máxima presentes en la columna 'Fecha' para conocer el rango temporal
print(f"Rango de fechas: {df['Fecha'].min()} a {df['Fecha'].max()}")

# 1. Indexar por Fecha

# Aseguramos que la columna 'Fecha' esté en formato datetime para operaciones temporales
df['Fecha'] = pd.to_datetime(df['Fecha'])
# Establecemos la columna 'Fecha' como índice del DataFrame; inplace=True modifica df directamente
df.set_index('Fecha', inplace=True)

# 2. Resampleo mensual

# Re-muestreamos la serie temporal por mes ('M') y sumamos los valores de la columna 'Ventas' por cada mes
monthly = df['Ventas'].resample('M').sum()
# Imprimimos un encabezado y las primeras filas del resultado mensual
print("\n=== VENTAS MENSUALES ===")
print(monthly.head())

# 3. Crecimiento YoY

# Calculamos el cambio porcentual interanual (year-over-year) usando 12 periodos (12 meses) y multiplicamos por 100
monthly_pct = monthly.pct_change(periods=12) * 100
# Imprimimos un encabezado y mostramos los primeros valores no nulos (dropna) del crecimiento YoY
print("\n=== CRECIMIENTO AÑO A AÑO (%) ===")
print(monthly_pct.dropna().head())

# 4. Resampleo semanal

# Re-muestreamos la serie temporal por semana ('W') y sumamos las ventas en cada semana
weekly = df['Ventas'].resample('W').sum()
# Imprimimos un encabezado y mostramos el inicio de la serie semanal
print("\n=== VENTAS SEMANALES ===")
print(weekly.head())

# 5. Detectar outliers (winsorize)

# Importamos la función winsorize desde scipy.stats.mstats para limitar extremos (winsorización)
from scipy.stats import mstats
# Aplicamos winsorize con límites del 5% en ambos extremos:
# los valores por debajo del percentil 5 se establecen en el percentil 5,
# y los valores por encima del percentil 95 se establecen en el percentil 95.
ventas_winsorized = mstats.winsorize(df['Ventas'], limits=0.05)
# Imprimimos un encabezado para la comparación de outliers
print(f"\n=== OUTLIERS ===")
# Mostramos el mínimo y máximo originales de la serie 'Ventas'
print(f"Valores originales - Min: {df['Ventas'].min():.2f}, Max: {df['Ventas'].max():.2f}")
# Mostramos el mínimo y máximo después de aplicar winsorize (para observar el efecto sobre extremos)
print(f"Valores winsorized - Min: {ventas_winsorized.min():.2f}, Max: {ventas_winsorized.max():.2f}")

# 6. Missing data (verificar si hay valores faltantes)

# Imprimimos un encabezado sobre missing data
print(f"\n=== MISSING DATA ===")
# Contamos e imprimimos cuántos valores NaN existen en la columna 'Ventas'
print(f"Valores faltantes en datos reales: {df['Ventas'].isna().sum()}")

# Si hubiera valores faltantes, usaríamos forward-fill (propagar el último valor observado hacia adelante)
if df['Ventas'].isna().sum() > 0:
    # Llenamos los NaN usando forward-fill y guardamos el resultado en df_filled (sin modificar df original)
    df_filled = df.fillna(method='ffill')
    # Mostramos cuántos NaN quedan después de aplicar forward-fill (esperamos 0 en la mayoría de los casos)
    print(f"Valores faltantes después de forward-fill: {df_filled['Ventas'].isna().sum()}")
else:
    # Si no hay NaN, informamos que los datos están completos
    print("No hay valores faltantes en los datos reales de Apple")

# 7. Resumen estadístico (descriptivo) del resample mensual

# Imprimimos un encabezado para el resumen estadístico
print("\n=== RESUMEN ESTADÍSTICO ===")
# Mostramos estadísticas descriptivas (count, mean, std, min, percentiles, max) de la serie mensual
print(monthly.describe())


---


**Datos financieros (acciones)**:

* Series OHLCV (Open/High/Low/Close/Volume).
* Cálculos útiles: retornos simples `pct_change()`, retornos logarítmicos `np.log(pct+1)`, volatilidad (std de retornos), drawdown, medias móviles (SMA/EMA), ratios (Sharpe).


---

### Enunciado del ejercicio

Una firma de **inversión cuantitativa** desea evaluar el desempeño de la acción de **Apple (AAPL)** durante el año 2023. Para ello, se requiere un análisis financiero que incluya **medidas de rentabilidad, riesgo, tendencia y gestión de drawdowns**, usando técnicas estándar de análisis de series temporales aplicadas en los mercados financieros.

El programa debe:

1. **Descargar los datos OHLCV (Open, High, Low, Close, Volume)** de la acción AAPL desde Yahoo Finance para el período del **1 de enero de 2023 al 31 de diciembre de 2023**.
2. **Calcular los retornos diarios** de dos formas:

   * Retorno simple (porcentaje de variación respecto al día anterior).
   * Retorno logarítmico (log(1 + retorno simple)).
     Mostrar los promedios de ambos tipos de retornos.
3. **Estimar la volatilidad anualizada**, utilizando la desviación estándar de los retornos diarios y asumiendo 252 días hábiles al año.
4. **Calcular indicadores de tendencia**:

   * Media móvil simple de 20 días (SMA-20).
   * Media móvil exponencial de 20 días (EMA-20).
     Mostrar el último valor de cada media y compararlo con el precio actual.
5. **Medir el drawdown máximo** en el período, es decir, la mayor caída porcentual respecto a un máximo histórico anterior.
6. **Calcular el ratio de Sharpe**, asumiendo una tasa libre de riesgo anual de 2%.

   * Exceso de retorno anualizado sobre la tasa libre de riesgo.
   * Volatilidad anualizada.
   * Ratio de Sharpe = exceso de retorno / volatilidad.
7. **Generar un resumen financiero final** que incluya:

   * Precio inicial y final del período.
   * Retorno total del año.
   * Volumen promedio negociado (número de acciones).

Este ejercicio permite aplicar conceptos fundamentales de **finanzas cuantitativas** (retornos, volatilidad, medias móviles, drawdown, Sharpe ratio) y conectarlos con la práctica en **análisis de datos en Python** utilizando `pandas`, `numpy` y `yfinance`.

---

In [ ]:
# Importamos pandas para manipular datos tabulares con DataFrame (lectura, transformación, agrupaciones, etc.)
import pandas as pd
# Importamos numpy para operaciones numéricas (arrays, funciones matemáticas, log, sqrt, etc.)
import numpy as np
# yfinance permite obtener datos de mercado desde Yahoo Finance de forma programática
import yfinance as yf

# Mensaje informativo en consola indicando inicio de descarga de datos OHLCV (Open/High/Low/Close/Volume)
print("Descargando datos OHLCV de Apple (AAPL)...")
# Creamos un objeto Ticker para "AAPL" que facilita la descarga de histórico y acceso a metadatos
ticker = yf.Ticker("AAPL")
# Descargamos el histórico de precios para el rango dado; devuelve un DataFrame con columnas OHLCV y un índice datetime
df = ticker.history(start="2023-01-01", end="2023-12-31")

# Imprimimos un encabezado para mostrar las primeras filas del DataFrame descargado
print("=== DATOS OHLCV (Open/High/Low/Close/Volume) ===")
# Mostramos las primeras 5 filas para inspección rápida de los datos descargados
print(df.head())

# ---------------------------------------------------------------------
# 1. RETORNOS SIMPLES Y LOGARÍTMICOS
# ---------------------------------------------------------------------

# Calculamos el retorno simple diario: porcentaje de cambio del precio de cierre respecto al día anterior
# pct_change() produce NaN en la primera fila porque no hay dato previo para comparar
df['returns'] = df['Close'].pct_change()
# Calculamos el retorno logarítmico diario: log(1 + retorno_simple)
# Los retornos logarítmicos son aditivos en el tiempo y se usan con frecuencia en finanzas
df['log_returns'] = np.log(1 + df['returns'])

# Imprimimos un encabezado para la sección de retornos
print("\n=== RETORNOS ===")
# Mostramos el retorno simple promedio de la serie (multiplicado por 100 para expresarlo en %)
# Nota: mean() ignorará los NaN (por ejemplo el primer elemento)
print(f"Retorno simple promedio: {df['returns'].mean()*100:.2f}%")
# Mostramos el retorno logarítmico promedio (también en %)
print(f"Retorno logarítmico promedio: {df['log_returns'].mean()*100:.2f}%")

# ---------------------------------------------------------------------
# 2. VOLATILIDAD (desviación estándar de retornos)
# ---------------------------------------------------------------------

# Calculamos la volatilidad anualizada:
# - df['returns'].std() es la desviación estándar diaria de los retornos
# - multiplicamos por sqrt(252) asumiendo ~252 días hábiles en un año para anualizar
volatilidad = df['returns'].std() * np.sqrt(252)  # Anualizada
# Imprimimos un encabezado y la volatilidad anualizada en porcentaje
print(f"\n=== VOLATILIDAD ===")
print(f"Volatilidad anualizada: {volatilidad*100:.2f}%")

# ---------------------------------------------------------------------
# 3. MEDIAS MÓVILES (SMA y EMA)
# ---------------------------------------------------------------------

# Calculamos la media móvil simple (SMA) de 20 días sobre el precio de cierre
# rolling(20).mean() produce NaN para las primeras 19 filas (ventana incompleta)
df['SMA_20'] = df['Close'].rolling(20).mean()
# Calculamos la media móvil exponencial (EMA) de 20 periodos; ewm() pondera más los precios recientes
df['EMA_20'] = df['Close'].ewm(span=20).mean()

# Imprimimos un encabezado y algunos KPIs: precio actual y últimos valores de SMA/EMA
print("\n=== MEDIAS MÓVILES ===")
# Precio actual (último precio de cierre disponible). Atención: si df está vacío, .iloc[-1] lanzará error.
print(f"Precio actual: ${df['Close'].iloc[-1]:.2f}")
# Último valor de SMA_20 (si hay suficientes datos; si no, será NaN)
print(f"SMA 20: ${df['SMA_20'].iloc[-1]:.2f}")
# Último valor de EMA_20
print(f"EMA 20: ${df['EMA_20'].iloc[-1]:.2f}")

# ---------------------------------------------------------------------
# 4. DRAWDOWN
# ---------------------------------------------------------------------

# Calculamos retornos acumulados: (1 + r_t) acumulado en el tiempo -> serie del valor relativo del capital
df['cum_returns'] = (1 + df['returns']).cumprod()
# Calculamos el máximo rodante (rolling max) de los retornos acumulados para identificar picos anteriores
df['rolling_max'] = df['cum_returns'].cummax()
# Drawdown = caída porcentual desde el máximo acumulado: (cum_returns / rolling_max) - 1
# Valores negativos representan caídas; 0 indica que estamos en el máximo histórico
df['drawdown'] = df['cum_returns'] / df['rolling_max'] - 1

# Obtenemos el drawdown máximo observado (el valor mínimo de la serie drawdown)
max_drawdown = df['drawdown'].min()
# Imprimimos el resultado del drawdown máximo en porcentaje
print(f"\n=== DRAWDOWN ===")
print(f"Máximo drawdown: {max_drawdown*100:.2f}%")

# ---------------------------------------------------------------------
# 5. RATIO DE SHARPE (asumiendo tasa libre de riesgo anual del 2%)
# ---------------------------------------------------------------------

# Definimos la tasa libre de riesgo (anual). En práctica se usa un activo sin riesgo (ej. T-bill).
risk_free_rate = 0.02
# Calculamos el retorno medio anualizado: mean diario * 252 - rf (exceso de retorno sobre rf)
# Aquí 'excess_returns' está en términos anuales (retorno promedio anual - rf)
excess_returns = df['returns'].mean() * 252 - risk_free_rate
# Calculamos el denominator: volatilidad anualizada = std(daily_returns) * sqrt(252)
# Ratio de Sharpe ≈ (retorno medio anual - rf) / volatilidad anualizada
sharpe_ratio = excess_returns / (df['returns'].std() * np.sqrt(252))

# Imprimimos el Ratio de Sharpe
# Nota: si la desviación estándar es 0 o hay NaN en returns, esto puede producir NaN o error de división por cero.
print(f"\n=== RATIO DE SHARPE ===")
print(f"Ratio de Sharpe: {sharpe_ratio:.2f}")

# ---------------------------------------------------------------------
# 6. RESUMEN FINANCIERO
# ---------------------------------------------------------------------

# Imprimimos un encabezado para el resumen final
print(f"\n=== RESUMEN FINANCIERO ===")
# Precio inicial: primer cierre disponible (puede ser NaN si df está vacío)
print(f"Precio inicial: ${df['Close'].iloc[0]:.2f}")
# Precio final: último cierre disponible
print(f"Precio final: ${df['Close'].iloc[-1]:.2f}")
# Retorno total en el período: (final / inicial - 1) * 100 para representarlo en %
print(f"Retorno total: {((df['Close'].iloc[-1]/df['Close'].iloc[0])-1)*100:.2f}%")
# Volumen promedio: media de la columna 'Volume' (representa número de acciones negociadas por periodo)
# Formateamos con separadores de miles; recordar que esto es volumen (unidades), no valor monetario.
print(f"Volumen promedio: {df['Volume'].mean():,.0f} acciones")


---

## 1.5 Selección de KPIs para dashboards financieros / ventas

**Ventas / Retail**:

* Ventas totales (periodo), tickets promedio, unidades vendidas, tasa de conversión (si hay visitas), margen bruto, ventas por categoría/producto, top-N productos.

**Financiero (acciones / portfolio)**:

* Precio actual, retorno diario / acumulado, volatilidad (σ), máximo drawdown, Sharpe ratio (exceso de retorno / volatilidad), Beta (requiere benchmark).

Fórmulas clave:

* YoY (%) = `(valor_t - valor_t-12m)/valor_t-12m * 100`
* Volatilidad anualizada ≈ `std(daily_returns) * sqrt(252)`
* Sharpe ≈ `(mean(daily_returns) * 252 - rf) / (std(daily_returns) * sqrt(252))` (rf = tasa libre de riesgo anual)

---


## 1.6 Mapas visuales y tipos de gráficas recomendadas

* Serie temporal → *Line*, *Area*, *Candlestick* (acciones).
* Comparación categorías → *Bar chart*, *Grouped bar*.
* Composición → *Stacked area / stacked bar* (evitar demasiadas categorías).
* Distribución → *Histogram*, *Boxplot*.
* Correlación → *Scatter* + *Heatmap* (matriz de correlación).
* Geografía → *Choropleth / Mapbox* (ventas por región).


---
### Enunciado del ejercicio

Una firma de **gestión de portafolios** solicita un análisis detallado del desempeño de **Apple (AAPL)** en el período **2022–2023**, comparándolo con su índice de referencia, el **S\&P 500**.

El objetivo es construir un **dashboard financiero interactivo en Python** que permita:

---

#### 1. Cálculo de KPIs Financieros

Se deben calcular y mostrar en consola los principales indicadores:

* **Precio actual** de la acción.
* **Retorno diario promedio (%):** variación promedio de los precios diarios.
* **Retorno total (%):** variación acumulada entre el primer y último precio del período.
* **Volatilidad anualizada (%):** medida de riesgo, calculada con la desviación estándar de los retornos.
* **Sharpe Ratio:** indicador de eficiencia riesgo–retorno, usando una tasa libre de riesgo del 2%.
* **Máximo drawdown (%):** pérdida máxima desde un pico hasta un valle en el período.
* **Beta:** sensibilidad de AAPL respecto al S\&P 500.
* **YoY (%):** retorno Year over Year (2022 vs 2023).

---

#### 2. Visualizaciones Interactivas con Plotly

El dashboard debe incluir las siguientes gráficas:

1. **Serie temporal (Line chart):** evolución del precio de AAPL.
2. **Candlestick chart:** representación OHLC (Open, High, Low, Close).
3. **Bar chart comparativo:** comparación de KPIs clave (Retorno Total, Volatilidad, Sharpe, Drawdown).
4. **Histograma de retornos diarios:** distribución de la variación porcentual diaria.
5. **Boxplot:** resumen de la dispersión de retornos diarios y detección de outliers.
6. **Scatter plot Apple vs S\&P 500:** relación de retornos entre AAPL y su benchmark.
7. **Matriz de correlación (Heatmap):** correlación entre los retornos de Apple y el S\&P 500.
8. **Area chart de Drawdown:** visualización de las caídas desde máximos históricos.

---

#### 3. Entregables

* **KPIs impresos en consola.**
* **8 visualizaciones interactivas**, cada una con un título claro y alineada con las métricas financieras estudiadas.
* Un **resumen de las visualizaciones** en consola indicando su propósito.

---

### Contexto Didáctico

Este ejercicio integra **Big Data financiero y visualización de datos interactivos**, permitiendo que los estudiantes:

* Conecten datos reales vía API (`yfinance`).
* Apliquen **conceptos de finanzas cuantitativas** (retornos, volatilidad, Sharpe, Beta, drawdown).
* Generen un **dashboard financiero** usando **Plotly** para comunicar los resultados a un cliente o comité de inversión.

---


In [ ]:
import pandas as pd       # importa la librería pandas y la asigna al alias 'pd' (se usa para manejar tablas como DataFrame)
import numpy as np        # importa la librería numpy y la asigna al alias 'np' (se usa para operaciones numéricas y funciones como sqrt)
import yfinance as yf     # importa yfinance y la asigna al alias 'yf' (se usa para descargar datos de mercado desde Yahoo Finance)
import plotly.express as px           # importa plotly.express como 'px' (API de alto nivel para crear gráficos interactivos fácilmente)
import plotly.graph_objects as go     # importa plotly.graph_objects como 'go' (API de bajo nivel para construir gráficos con más control, p.ej. candlesticks)
import plotly.io as pio   # importa plotly.io como 'pio' (permite configurar renderizadores y guardar gráficos)
import time               # importa el módulo time (se usará para pausar la ejecución temporalmente con time.sleep)
import threading          # importa threading (se usará para ejecutar funciones en hilos y así implementar timeouts)


# Configuración
pio.renderers.default = "browser"  # asigna el renderizador por defecto a "browser", lo que hace que fig.show() abra el gráfico en el navegador


def ejecutar_con_timeout(func, timeout=10, nombre="gráfico"):  # define función que recibe: func (callable), timeout en segundos, y nombre (texto para mensajes)
    """Ejecuta función con timeout para evitar cuelgues"""      # docstring: breve descripción de propósito de la función
    resultado = None                                            # inicializa variable donde se guardará el resultado devuelto por func() si termina correctamente
    error = None                                                # inicializa variable donde se guardará la excepción si ocurre durante la ejecución
    
    def target():                                               # define función interna que será ejecutada dentro de un hilo
        nonlocal resultado, error                               # declara que 'resultado' y 'error' referencian las variables del scope exterior y se pueden modificar
        try:
            resultado = func()                                  # intenta ejecutar la función pasada y asigna su valor a 'resultado'
        except Exception as e:                                  # captura cualquier excepción que ocurra durante la ejecución
            error = e                                           # guarda la excepción en 'error' para manejarla fuera del hilo
    
    thread = threading.Thread(target=target)                    # crea un objeto Thread que ejecutará la función 'target' en paralelo
    thread.daemon = True                                        # marca el hilo como daemon: no impedirá que el programa principal termine si el hilo sigue activo
    thread.start()                                              # inicia la ejecución del hilo (lanza target() en segundo plano)
    thread.join(timeout)                                        # bloquea la ejecución actual hasta que el hilo termine o hasta que pasen 'timeout' segundos (lo que ocurra primero)
    
    if thread.is_alive():                                       # comprueba si el hilo sigue vivo después del join con timeout (si sigue, no terminó a tiempo)
        print(f"⚠ Timeout en {nombre} - Continuando...")       # avisa que la función excedió el tiempo permitido
        return False                                            # devuelve False indicando fallo por timeout (nota: el hilo puede seguir corriendo en background)
    elif error:                                                 # si hubo una excepción capturada dentro del hilo
        print(f"⚠ Error en {nombre}: {error}")                  # muestra el mensaje de error (la excepción guardada en 'error')
        return False                                            # devuelve False indicando fallo por excepción
    else:
        return True                                             # si no hubo error ni timeout, devuelve True indicando éxito


def crear_grafico(func_generacion, archivo, nombre, timeout=15):  # define función que genera, muestra y guarda un gráfico: recibe una función generadora, ruta de archivo, nombre descriptivo y timeout
    """Crea gráfico de forma segura con timeout"""                # docstring: indica que la función usa timeout para evitar bloqueos
    def crear_y_mostrar():                                        # función interna que encapsula la generación y guardado del gráfico
        fig = func_generacion()                                   # llama a la función que debe devolver un objeto figura de Plotly y lo asigna a 'fig'
        fig.show()                                                # muestra la figura en el renderizador configurado (aquí: abre en el navegador)
        fig.write_html(archivo)                                   # guarda la figura en disco como archivo HTML en la ruta especificada por 'archivo'
        return fig                                                # devuelve la figura creada por si se quiere usar el valor retornado
    
    if ejecutar_con_timeout(crear_y_mostrar, timeout, nombre):    # ejecuta crear_y_mostrar() usando la función ejecutar_con_timeout para protegerla con timeout
        print(f"✓ {nombre} generado correctamente")               # si ejecutar_con_timeout devolvió True, imprime confirmación de éxito
        return True                                               # devuelve True indicando que el gráfico se creó y guardó correctamente
    return False                                                  # si ejecutar_con_timeout devolvió False, retorna False indicando que la creación falló


# === DATOS ===
print("Descargando datos de Apple y S&P 500...")                   # imprime en consola para informar al usuario que iniciará la descarga de datos
aapl = yf.Ticker("AAPL").history(start="2022-01-01", end="2023-12-31")  # usa yfinance: crea un objeto Ticker para el símbolo "AAPL" y descarga su historial entre las fechas dadas; devuelve un DataFrame con columnas Open/High/Low/Close/Volume/Dividends/Stock Splits y un índice de fechas (DatetimeIndex)
sp500 = yf.Ticker("^GSPC").history(start="2022-01-01", end="2023-12-31") # descarga el historial del índice S&P 500 (símbolo ^GSPC) en el mismo rango; también devuelve DataFrame con cierre y demás columnas


# === CÁLCULOS KPIs ===
returns = aapl['Close'].pct_change().dropna()                      # toma la columna 'Close' del DataFrame de Apple, calcula el cambio porcentual entre filas consecutivas (pct_change() -> (current/prev)-1), y dropna() elimina el primer NaN resultante del cálculo
sp500_returns = sp500['Close'].pct_change().dropna()               # hace lo mismo para el S&P 500: cálculos de retornos diarios en formato decimal (ej. 0.01 = 1%)


# KPIs básicos
precio_actual = aapl['Close'].iloc[-1]                             # obtiene el último valor de la columna 'Close' usando iloc[-1] (última fila del DataFrame) -> precio de cierre más reciente disponible
retorno_diario = returns.mean() * 100                              # calcula la media aritmética de los retornos diarios (returns.mean()) y lo multiplica por 100 para expresarlo como porcentaje medio diario
retorno_total = (aapl['Close'].iloc[-1] / aapl['Close'].iloc[0] - 1) * 100  
# calcula el retorno total sobre el periodo: (precio_final / precio_inicial - 1) * 100 => porcentaje de crecimiento desde la primera hasta la última fecha del DataFrame
volatilidad = returns.std() * np.sqrt(252) * 100                   # calcula volatilidad anualizada: desviación estándar diaria (returns.std()) * sqrt(252 dias hábiles) * 100 para expresarlo en %; nota: pandas por defecto usa ddof=1 en std (desviación muestral)


# KPIs avanzados
sharpe = (returns.mean() * 252 - 0.02) / (returns.std() * np.sqrt(252))  
# calcula Sharpe ratio anualizado: (retorno medio diario * 252 - tasa_libre_de_riesgo_anual) / (desviación estándar diaria * sqrt(252))
# en este código se asume tasa libre de riesgo = 0.02 (2% anual); returns.mean()*252 annualiza el promedio diario

cum_returns = (1 + returns).cumprod()                               # calcula los retornos acumulados multiplicativos día a día: (1+retorno1)*(1+retorno2)*... -> Series con índice de fechas que muestra crecimiento acumulado desde el inicio
rolling_max = cum_returns.cummax()                                  # calcula en cada punto el máximo histórico alcanzado por cum_returns hasta esa fecha (se usa para medir caídas desde picos)
drawdown = (cum_returns / rolling_max - 1) * 100                     # calcula el drawdown en % como la pérdida relativa al máximo histórico hasta el momento: (cum_returns / rolling_max - 1) * 100
max_drawdown = drawdown.min()                                       # extrae el valor mínimo de la serie drawdown: el drawdown máximo (valor más negativo) observado en el periodo
beta = returns.cov(sp500_returns) / sp500_returns.var()              # calcula Beta: covarianza entre retornos de Apple y retornos del S&P (returns.cov) dividida por la varianza de los retornos del S&P; da la sensibilidad de Apple frente al mercado
yoy = (aapl['Close'].iloc[-1] - aapl['Close'].iloc[0]) / aapl['Close'].iloc[0] * 100  
# calcula YoY (Year-over-Year aquí definido como cambio desde inicio hasta fin del periodo) como (precio_final - precio_inicial) / precio_inicial * 100 para %


# Mostrar KPIs
print("\n=== KPIs FINANCIEROS ===")                                   # imprime un encabezado separador en la consola
print(f"Precio actual: ${precio_actual:.2f}")                         # imprime el precio actual formateado con 2 decimales; utiliza f-string y formato :.2f
print(f"Retorno diario promedio: {retorno_diario:.3f}%")              # imprime el retorno diario promedio con 3 decimales y símbolo %
print(f"Retorno total: {retorno_total:.1f}%")                         # imprime el retorno total con 1 decimal y %
print(f"Volatilidad anualizada: {volatilidad:.1f}%")                  # imprime la volatilidad anualizada con 1 decimal y %
print(f"Sharpe ratio: {sharpe:.2f}")                                  # imprime el Sharpe ratio con 2 decimales
print(f"Máximo drawdown: {max_drawdown:.1f}%")                         # imprime el máximo drawdown con 1 decimal y %
print(f"Beta (vs S&P 500): {beta:.2f}")                                # imprime la beta con 2 decimales
print(f"YoY (2022-2023): {yoy:.1f}%")                                  # imprime la variación YoY en % con 1 decimal


# === VISUALIZACIONES ===
print("\n=== GENERANDO GRÁFICOS ===")                                   # imprime encabezado indicando que inicia generación de gráficos
graficos_exitosos = 0                                                   # inicializa contador en 0 para llevar registro de gráficos creados correctamente

# 1. Precio temporal
def grafico_precio():                                                   # define función que generará el gráfico de precios (no la ejecuta todavía)
    return px.line(aapl, x=aapl.index, y='Close', title='Precio Apple 2022-2023')
    # usa px.line para crear figura: 
    # - primer argumento es el DataFrame 'aapl' (opcionalmente se podía pasar solo arrays),
    # - x=aapl.index especifica que el eje x será el índice del DataFrame (fechas),
    # - y='Close' indica que la serie a graficar es la columna Close,
    # - title fija el título del gráfico; la función devuelve el objeto figura de Plotly

if crear_grafico(grafico_precio, "precio_apple.html", "Gráfico de precio"):  # llama a crear_grafico pasando la función generadora (grafico_precio), nombre de archivo y etiqueta descriptiva
    graficos_exitosos += 1                                               # si crear_grafico devolvió True, incrementa contador de gráficos exitosos
time.sleep(2)                                                             # pausa la ejecución 2 segundos para dar tiempo al usuario o evitar sobrecarga al abrir múltiples pestañas

# 2. Candlestick
def grafico_velas():                                                     # define función que genera un gráfico de velas (candlestick)
    fig = go.Figure(data=go.Candlestick(x=aapl.index, open=aapl['Open'], 
                                       high=aapl['High'], low=aapl['Low'], close=aapl['Close']))
    # construye un objeto Candlestick con:
    # - x: serie de fechas (índice),
    # - open/high/low/close: columnas correspondientes del DataFrame 'aapl';
    # go.Candlestick devuelve un trace que se incrusta en una Figure()
    fig.update_layout(title='Gráfico de Velas - Apple')                  # modifica el layout de la figura para establecer el título
    return fig                                                            # devuelve la figura lista para mostrarse

if crear_grafico(grafico_velas, "velas_apple.html", "Gráfico de velas"):  # intenta crear, mostrar y guardar el gráfico de velas con timeout
    graficos_exitosos += 1                                               # incrementa contador si fue exitoso
time.sleep(2)                                                             # espera 2 segundos antes del siguiente gráfico

# 3. KPIs comparativo
def grafico_kpis():                                                       # define función que genera un gráfico de barras con los KPIs seleccionados
    kpis_data = pd.DataFrame({
        'Métrica': ['Retorno Total', 'Volatilidad', 'Sharpe', 'Max Drawdown'], 
        'Valor': [retorno_total, volatilidad, sharpe, max_drawdown]
    })
    # crea un DataFrame 'kpis_data' con dos columnas: 'Métrica' (labels) y 'Valor' (valores numéricos);
    # las listas deben tener la misma longitud; este DataFrame será usado por px.bar para dibujar barras
    return px.bar(kpis_data, x='Métrica', y='Valor', title='KPIs Financieros Apple')  
    # crea y devuelve un gráfico de barras donde el eje x son las métricas y el eje y sus valores

if crear_grafico(grafico_kpis, "kpis_apple.html", "Gráfico de KPIs"):
    graficos_exitosos += 1
time.sleep(2)

# 4. Histograma retornos
def grafico_histograma():                                                 # define función que genera histograma de retornos diarios
    returns_clean = (returns * 100)[(returns * 100 >= -10) & (returns * 100 <= 10)]
    # multiplica returns por 100 para convertir de fracción a porcentaje y luego filtra filas cuyo valor esté entre -10% y +10%
    # esto elimina outliers extremos del histograma para una visualización más clara
    fig = px.histogram(x=returns_clean, title='Distribución Retornos Diarios (%)', nbins=20)
    # crea un histograma con los datos limpios; nbins=20 define 20 bins (barras del histograma)
    fig.update_layout(showlegend=False)                                   # quita la leyenda del gráfico (no necesaria para un histograma simple)
    return fig                                                            # devuelve la figura

if crear_grafico(grafico_histograma, "histograma_retornos.html", "Histograma de retornos"):
    graficos_exitosos += 1
time.sleep(2)

# 5. Boxplot
def grafico_boxplot():                                                     # define función para boxplot de retornos
    fig = px.box(y=returns*100, title='Boxplot Retornos Diarios (%)')     # crea un boxplot con los retornos convertidos a % (returns*100)
    fig.update_layout(showlegend=False)                                   # quita la leyenda para simplificar la visualización
    return fig                                                            # devuelve la figura

if crear_grafico(grafico_boxplot, "boxplot_retornos.html", "Boxplot de retornos"):
    graficos_exitosos += 1
time.sleep(2)

# 6. Correlación
aapl_aligned = aapl['Close'].pct_change().dropna()                         # recalcula retornos de Apple (similar a 'returns'); se usa para asegurar alineamiento temporal independiente
sp500_aligned = sp500['Close'].pct_change().dropna()                       # recalcula retornos del S&P500
common_dates = aapl_aligned.index.intersection(sp500_aligned.index)        # calcula el índice de fechas que están en ambas series (intersección de índices)
aapl_aligned = aapl_aligned.loc[common_dates]                              # filtra la serie de Apple manteniendo únicamente las fechas comunes
sp500_aligned = sp500_aligned.loc[common_dates]                            # filtra la serie del S&P 500 para tener exactamente las mismas fechas en ambos

correlation_data = pd.DataFrame({
    'Apple': aapl_aligned,
    'S&P 500': sp500_aligned
}).dropna()
# crea un DataFrame con dos columnas ('Apple' y 'S&P 500') usando las series alineadas por fecha; dropna() elimina filas con valores faltantes

def grafico_scatter():                                                      # define función para gráfico de dispersión entre retornos de Apple y S&P 500
    return px.scatter(correlation_data, x='S&P 500', y='Apple', 
                     title='Correlación Apple vs S&P 500')                  # crea un scatter plot con S&P en eje X y Apple en eje Y; la relación visual da una idea de correlación

if crear_grafico(grafico_scatter, "correlacion_scatter.html", "Gráfico de correlación"):
    graficos_exitosos += 1
time.sleep(2)

# 7. Heatmap correlación
def grafico_heatmap():                                                      # define función para generar un heatmap (matriz de correlación)
    corr_matrix = correlation_data.corr()                                   # calcula la matriz de correlación entre las columnas del DataFrame (Pearson por defecto)
    return px.imshow(corr_matrix, text_auto=True, 
                    title='Matriz de Correlación', color_continuous_scale='RdBu')
    # px.imshow dibuja la matriz como imagen coloreada; text_auto=True escribe los valores numéricos en cada celda;
    # color_continuous_scale='RdBu' aplica una paleta divergente (rojo-azul) para resaltar correlaciones positivas/negativas

if crear_grafico(grafico_heatmap, "heatmap_correlacion.html", "Matriz de correlación"):
    graficos_exitosos += 1
time.sleep(2)

# 8. Drawdown
def grafico_drawdown():                                                     # define función para graficar drawdown (caída desde máximos)
    drawdown_aligned = drawdown.reindex(aapl.index).fillna(0)               # reindexa la serie 'drawdown' para que tenga el mismo índice (fechas) que 'aapl'; rellena valores faltantes con 0
    return px.area(x=aapl.index, y=drawdown_aligned, title='Drawdown Apple')# crea un gráfico de área con el drawdown alineado respecto al índice de fechas de Apple

if crear_grafico(grafico_drawdown, "drawdown_apple.html", "Gráfico de drawdown"):
    graficos_exitosos += 1

# === RESUMEN ===
print(f"\n=== RESUMEN ===")                             # imprime título de sección resumen en consola con un salto de línea inicial
print(f"Gráficos generados: {graficos_exitosos}/8")    # imprime la cantidad de gráficos que se generaron correctamente sobre el total esperado (8)
print("Archivos HTML creados:")                        # imprime encabezado de la lista de archivos guardados
print("- precio_apple.html")                           # lista el archivo generado para el gráfico de precio
print("- velas_apple.html")                            # lista el archivo generado para el gráfico de velas
print("- kpis_apple.html")                             # lista el archivo generado para el gráfico de KPIs
print("- histograma_retornos.html")                    # lista el archivo generado para el histograma de retornos
print("- boxplot_retornos.html")                       # lista el archivo generado para el boxplot
print("- correlacion_scatter.html")                    # lista el archivo generado para el scatter de correlación
print("- heatmap_correlacion.html")                    # lista el archivo generado para el heatmap de correlación
print("- drawdown_apple.html")                         # lista el archivo generado para el gráfico de drawdown


---

## 1.7 Interactividad y patrones de callbacks en Dash

Patrones comunes:

* **Dropdown → Update Graph** (filtrado por producto/región).
* **DatePickerRange → Resample / Update**.
* **Multi-output callbacks** (actualizar varios gráficos desde el mismo input).
* **State** para acciones que requieren confirmación (por ejemplo, botón "Actualizar").
* **Caching** para llamadas costosas (APIs, cálculos).
* **Client-side callbacks** para cálculos sencillos y mejorar latencia.


### Enunciado del Ejercicio: **Dashboard Interactivo con Datos Financieros Reales (Apple y Microsoft)**

En este ejercicio se propone la construcción de un **dashboard interactivo** utilizando la librería **Dash** en Python. El objetivo es mostrar cómo integrar datos financieros reales en una aplicación web que permita a los usuarios explorar información a través de controles dinámicos.

El sistema se conecta en tiempo real a la API de **Yahoo Finance** mediante la librería `yfinance`, descargando datos históricos de las empresas **Apple (AAPL)** y **Microsoft (MSFT)** durante el año 2023.

Una vez descargados, los datos son transformados y organizados para simular un escenario de **ventas financieras** a partir del precio de cierre de cada acción. Posteriormente, estos datos se integran en un dashboard con las siguientes características:

1. **Selección de empresa**

   * A través de un **menú desplegable (`Dropdown`)**, el usuario puede seleccionar entre Apple o Microsoft.

2. **Selección de rango de fechas**

   * Con un **selector de rango de fechas (`DatePickerRange`)**, el usuario puede filtrar la información para analizar períodos específicos dentro del año 2023.

3. **Botón de actualización**

   * Un botón permite actualizar manualmente el estado del dashboard y registrar cuántas veces se ha realizado una actualización.

4. **Gráfico de series temporales**

   * Se muestra un **gráfico de líneas (`Line Chart`)** que representa la evolución del precio de las acciones (simulado como ventas) a lo largo del tiempo.

5. **Indicador clave de desempeño (KPI)**

   * Se presenta el **precio promedio** de las acciones dentro del rango de fechas seleccionado.

6. **Estado de actualización**

   * El dashboard informa al usuario cada vez que se actualizan los datos mediante un mensaje dinámico que incluye el número de clics realizados en el botón de actualización.

---

In [ ]:
import dash                                     # Importa Dash (framework web para construir dashboards en Python)
from dash import dcc, html, Input, Output, State, callback
# ↑ Importa componentes de Dash:
#    - dcc: componentes interactivos (Graphs, Dropdowns, DatePicker, etc.)
#    - html: componentes HTML (Div, H3, Button, etc.)
#    - Input/Output/State: tipos usados en los callbacks para declarar entradas/salidas/estado
#    - callback: decorador para registrar funciones callback (alternativa a app.callback)

import plotly.express as px                     # plotly.express: API sencilla para crear gráficos interactivos
import pandas as pd                             # pandas: manipulación de datos tabulares (DataFrame)
import yfinance as yf                           # yfinance: descarga de datos financieros desde Yahoo Finance

# Crear app y obtener datos reales
app = dash.Dash(__name__)                       # Instancia la aplicación Dash; __name__ ayuda en la resolución de recursos

# Datos reales de Apple y Microsoft
print("Descargando datos reales...")            # Mensaje informativo que se mostrará en la consola al ejecutar
aapl = yf.Ticker("AAPL").history(start="2023-01-01", end="2023-12-31")
# ↑ Descarga el histórico OHLCV de Apple entre las fechas indicadas; devuelve un DataFrame con índice datetime
msft = yf.Ticker("MSFT").history(start="2023-01-01", end="2023-12-31")
# ↑ Descarga el histórico OHLCV de Microsoft (MSFT) en el mismo rango

# Preparar datos para el dashboard
aapl_data = aapl.reset_index()                   # Convierte el índice de fecha en una columna 'Date' (DataFrame "plano")
aapl_data['Empresa'] = 'Apple'                   # Crea columna 'Empresa' con valor 'Apple' para identificar la serie
aapl_data['Ventas'] = aapl_data['Close']         # Crea columna 'Ventas' copiando el precio de cierre (simulación de ventas)

msft_data = msft.reset_index()                   # Mismo procedimiento para Microsoft
msft_data['Empresa'] = 'Microsoft'               # Etiqueta la compañía
msft_data['Ventas'] = msft_data['Close']         # Usa Close como "Ventas" también

# Combinar datos
df = pd.concat([aapl_data[['Date', 'Empresa', 'Ventas']], 
                msft_data[['Date', 'Empresa', 'Ventas']]], ignore_index=True)
# ↑ Concatena verticalmente las filas de Apple y Microsoft en un solo DataFrame; ignore_index=True reindexa filas
df.columns = ['Fecha', 'Región', 'Ventas']       # Renombra columnas a 'Fecha','Región','Ventas' (se usa 'Región' por compatibilidad con ejemplos previos)

# Layout compacto
app.layout = html.Div([                           # Contenedor principal del layout (estructura de la página)
    html.H3("Dashboard - Patrones de Callbacks"),# Título H3 del dashboard

    # Controles
    dcc.Dropdown(id='region_dropdown', options=[  # Dropdown para seleccionar la "Región" (aquí usada para seleccionar empresa)
        {'label': 'Apple', 'value': 'Apple'},     # Opción: Apple
        {'label': 'Microsoft', 'value': 'Microsoft'} # Opción: Microsoft
    ], value='Apple'),                            # Valor por defecto: 'Apple'

    dcc.DatePickerRange(id='date_picker',         # Selector de rango de fechas
                       start_date=df['Fecha'].min(), # Fecha de inicio por defecto: la mínima fecha del DataFrame
                       end_date=df['Fecha'].max()),  # Fecha final por defecto: la máxima fecha del DataFrame

    html.Button('Actualizar', id='update_button', n_clicks=0), # Botón que incrementa n_clicks cada vez que se presiona

    # Outputs
    html.Div(id='kpi_total'),                      # Div vacío que recibirá texto (KPI) desde un callback
    dcc.Graph(id='ventas_graph'),                  # Componente gráfico que mostrará la serie de ventas/precio
    html.Div(id='update_status')                   # Div para mostrar estado de actualización (mensaje)
])

# === PATRONES DE CALLBACKS ===

# 1. Multi-output callback
@callback(
    [Output('ventas_graph', 'figure'), Output('kpi_total', 'children')],
    [Input('region_dropdown', 'value'),
     Input('date_picker', 'start_date'),
     Input('date_picker', 'end_date')]
)
# ↑ Decorador que registra la función update_graphs_and_kpis como callback:
#    - Tiene dos outputs: la figura del gráfico y el contenido (children) del KPI.
#    - Se dispara cuando cambia el valor del dropdown o el rango de fechas.
def update_graphs_and_kpis(region, start_date, end_date):
    # Filtramos el DataFrame por la "Región" seleccionada (empresa) y el rango de fechas proporcionado
    filtered_df = df[
        (df['Región'] == region) & 
        (df['Fecha'] >= start_date) & 
        (df['Fecha'] <= end_date)
    ]
    # ↑ Aquí se aplican tres máscaras booleans:
    #    - df['Región'] == region -> filas de la empresa elegida
    #    - df['Fecha'] >= start_date -> fecha mayor o igual al inicio (DatePickerRange devuelve strings o datetime)
    #    - df['Fecha'] <= end_date -> fecha menor o igual al fin
    # Nota: si start_date/end_date son strings, pandas realiza comparación con Timestamp; en algunos casos puede hacerse pd.to_datetime()

    fig = px.line(filtered_df, x='Fecha', y='Ventas', title=f'Precio de Acciones - {region}')
    # ↑ Crea un gráfico de línea con Plotly Express usando la columna 'Fecha' en el eje x y 'Ventas' en el eje y
    total = filtered_df['Ventas'].mean()            # Calcula el precio/venta promedio en el período filtrado (KPI simple)

    return fig, f"Precio Promedio: ${total:.2f}"    # Devuelve la figura y un string formateado (multi-output)

# 2. Callback con State
@callback(
    Output('update_status', 'children'),
    [Input('update_button', 'n_clicks')],
    [State('region_dropdown', 'value')]
)
# ↑ Decorador para el callback que actualiza un estado/etiqueta de status:
#    - Se dispara cuando 'update_button' es clickeado (Input).
#    - Usa State para leer el valor actual del dropdown sin que éste dispare el callback.
def update_status(n_clicks, region):
    if n_clicks > 0:                                # Si el botón ha sido presionado al menos una vez
        return f"✅ Actualizado {region} (Click #{n_clicks})"  # Devuelve un mensaje con la cantidad de clicks y la región
    return ""                                       # Si no se ha clickeado, retorna cadena vacía (sin mensaje)

if __name__ == '__main__':
    app.run(debug=True)                             # Ejecuta la app en modo debug (servidor local, recarga automática)


---

## 1.8 Rendimiento y escalabilidad

* **Downsample**: no enviar 100k puntos al navegador; agrupa/reesamplea en el servidor.
* **Cache** respuestas de APIs (ej. `flask_caching`).
* **Store** data intermedia en `dcc.Store` (útil para evitar recálculos).
* **Paginar** tablas grandes.
* **Web worker / Background tasks** para cálculos pesados (Celery si es necesario).

---



## 1.9 Seguridad y límites

* Nunca exponer API keys en el frontend; use variables de entorno y un backend seguro.
* Ten en cuenta límites de uso de APIs (Alpha Vantage tiene límites en la versión gratuita). ([alphavantage.co][2])
* Cuidado con datos sensibles de clientes (anonimizar/agregar).

---


## 1.10 Despliegue básico

* Dockerizar la app, usar Gunicorn como servidor WSGI y nginx como proxy.
* Configurar logging, health checks y límites de memoria.
* Para demo o uso académico: Heroku / Render / Vercel (si el stack lo permite) o desplegar en una VM en la nube.

---


# 2. PRÁCTICA — Ejercicios con datasets descargables


## Ejercicio 1 — Dashboard de precios de acciones (caso: comité de inversión)

**Contexto:** El comité quiere un dashboard para monitorizar el comportamiento de una acción (ej. AAPL) y compararla con su benchmark. Debe mostrar precios OHLC, volumen, medias móviles, retornos acumulados, drawdown y KPIs (retorno desde X fecha, volatilidad anual).

**Dataset / cómo obtenerlo:**

* Opción manual: desde la pestaña *Historical Data* de Yahoo Finance (seleccionar rango y *Download* → CSV). ([macroption.com][1])
* Opción programática (recomendada en clase): `yfinance` para Python (no requiere API key), o Alpha Vantage API si prefieres CSV/JSON y control de rate limits. ([alphavantage.co][2])

**Instalación requerida:**

```bash
pip install dash dash-bootstrap-components pandas yfinance plotly flask-caching
```



**Código completo (app\_stock.py)** — *ejemplo listo para ejecutar*:



In [ ]:
"""
app_stock.py
Dashboard interactivo para precios de acciones (yfinance + Dash)
"""

from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from flask_caching import Cache
import numpy as np

# --- Configuración app ---
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

# Cache simple (en memoria) para evitar múltiples llamadas a API
cache = Cache(app.server, config={'CACHE_TYPE': 'SimpleCache'})
CACHE_TIMEOUT = 600  # segundos

@cache.memoize(timeout=CACHE_TIMEOUT)
def fetch_stock_data(ticker, start, end):
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty:
        return None
    df.reset_index(inplace=True)
    return df.to_json(date_format='iso', orient='split')

def compute_indicators(df):
    df['SMA20'] = df['Close'].rolling(window=20).mean()
    df['SMA50'] = df['Close'].rolling(window=50).mean()
    df['returns'] = df['Close'].pct_change()
    df['vol_daily'] = df['returns'].rolling(20).std()
    df['vol_annual'] = df['vol_daily'] * np.sqrt(252)
    df['cum_returns'] = (1 + df['returns']).cumprod()
    df['rolling_max'] = df['cum_returns'].cummax()
    df['drawdown'] = df['cum_returns'] / df['rolling_max'] - 1
    return df

# --- Layout ---
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H2("Dashboard: Precios de Acciones (Ej. AAPL)"), className='my-2')),
    dbc.Row([
        dbc.Col([
            dbc.Label("Ticker"),
            dcc.Input(id='input_ticker', type='text', value='AAPL', debounce=True),
            dbc.Label("Rango de fechas"),
            dcc.DatePickerRange(
                id='date_picker',
                start_date=pd.to_datetime('2023-01-01'),
                end_date=pd.to_datetime('today')
            ),
            dbc.Checklist(
                options=[
                    {"label":"SMA 20","value":"SMA20"},
                    {"label":"SMA 50","value":"SMA50"}
                ],
                value=["SMA20"],
                id="ma_checklist",
                inline=True
            ),
            html.Br(),
            dbc.Button("Actualizar", id="btn_update", color="primary")
        ], width=3),

        dbc.Col([
            dbc.Row([
                dbc.Col(dbc.Card([dbc.CardBody([html.H6("Precio Último"), html.H4(id='kpi_price')])]), width=3),
                dbc.Col(dbc.Card([dbc.CardBody([html.H6("Retorno desde inicio"), html.H4(id='kpi_return')])]), width=3),
                dbc.Col(dbc.Card([dbc.CardBody([html.H6("Vol. anual (σ)"), html.H4(id='kpi_vol')])]), width=3),
                dbc.Col(dbc.Card([dbc.CardBody([html.H6("Max Drawdown"), html.H4(id='kpi_dd')])]), width=3),
            ]),
            dcc.Loading(dcc.Graph(id='price_chart'), type='default'),
            dcc.Loading(dcc.Graph(id='volume_chart'), type='circle')
        ], width=9)
    ])
], fluid=True)

# --- Callbacks ---
@app.callback(
    [Output('price_chart','figure'),
     Output('volume_chart','figure'),
     Output('kpi_price','children'),
     Output('kpi_return','children'),
     Output('kpi_vol','children'),
     Output('kpi_dd','children')],
    [Input('btn_update','n_clicks')],
    [State('input_ticker','value'), State('date_picker','start_date'),
     State('date_picker','end_date'), State('ma_checklist','value')]
)
def update_dashboard(n_clicks, ticker, start, end, ma_values):
    if not ticker:
        return go.Figure(), go.Figure(), "—", "—", "—", "—"
    raw = fetch_stock_data(ticker.upper(), start, end)
    if raw is None:
        return go.Figure(), go.Figure(), "No data", "No data", "No data", "No data"
    df = pd.read_json(raw, orient='split')
    df = compute_indicators(df)

    # Price chart (candlestick)
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=df['Date'], open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='OHLC'))
    if 'SMA20' in ma_values:
        fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA20'))
    if 'SMA50' in ma_values:
        fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA50'))
    fig.update_layout(title=f"{ticker.upper()} - OHLC", xaxis_rangeslider_visible=False)

    # Volume chart
    fig_vol = go.Figure([go.Bar(x=df['Date'], y=df['Volume'])])
    fig_vol.update_layout(title="Volumen")

    # KPIs
    last_price = f"${df['Close'].iloc[-1]:.2f}"
    total_return = f"{((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100:.2f}%"
    vol_ann = f"{df['vol_annual'].iloc[-1]*100:.2f}%"
    dd = f"{df['drawdown'].min()*100:.2f}%"

    return fig, fig_vol, last_price, total_return, vol_ann, dd

if __name__ == "__main__":
    app.run(debug=True, port=8050)


**Explicación:**

* Uso de `yfinance` para evitar gestión de API keys en la práctica.
* Caching con `flask_caching` para reducir llamadas.
* Cálculo de SMA, volatilidad anual y drawdown.
* Diseño del layout con `dash_bootstrap_components`.



---
**Actividad adicional**

1. Añadir *RSI* y permitir mostrar/ocultar indicadores técnicos.
2. Comparar el ticker con un benchmark (ej. ^GSPC) en la misma vista.
3. Implementar export CSV para los datos filtrados.

---

## Ejercicio 2 — Dashboard de ventas retail (caso: gerente de tienda)

**Contexto:** El gerente necesita ver ventas por tienda, producto y campaña. Quiere identificar top 10 productos, evolución semanal y comparar zonas.

**Dataset / cómo obtenerlo:**

* Usa un dataset de retail disponible en Kaggle (hay múltiples: `retaildataset`, `retail-sales-data`, etc. — descargar vía web o Kaggle API). ([kaggle.com][4])

**Cómo descargar (Kaggle):**

1. Crear cuenta en Kaggle.
2. Generar API token (archivo `kaggle.json`).
3. En terminal: `kaggle datasets download -d <owner>/<dataset>` y `unzip`.

**Código esencial (esqueleto app\_sales.py):**



In [ ]:
# app_sales.py (esqueleto)
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Crear datos de ejemplo (ventas retail)
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)
fechas = pd.date_range('2023-01-01', '2023-12-31', freq='D')
regiones = ['Norte', 'Sur', 'Este', 'Oeste']
productos = ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E']
tiendas = ['Tienda 1', 'Tienda 2', 'Tienda 3']

data = []
for fecha in fechas:
    for region in regiones:
        for producto in productos:
            for tienda in tiendas:
                if np.random.random() > 0.3:  # 70% probabilidad de venta
                    ventas = np.random.normal(1000, 200)
                    transacciones = np.random.randint(1, 10)
                    unidades = np.random.randint(1, 20)
                    margen = np.random.uniform(0.1, 0.4)
                    
                    data.append({
                        'date': fecha,
                        'region': region,
                        'product': producto,
                        'store': tienda,
                        'sales': max(0, ventas),
                        'transactions': transacciones,
                        'units': unidades,
                        'margin': margen
                    })

df = pd.DataFrame(data)
df['week'] = df['date'].dt.to_period('W').apply(lambda r: r.start_time)

# layout: filtros region/producto/date range + KPIs + 3 gráficas
app.layout = dbc.Container([
    html.H3("Dashboard de Ventas - Retail"),
    dbc.Row([
        dbc.Col([dcc.Dropdown(id='region_drop', options=[{'label':r,'value':r} for r in df['region'].unique()], value=None, placeholder="Todas")], width=3),
        dbc.Col([dcc.Dropdown(id='product_drop', options=[{'label':p,'value':p} for p in df['product'].unique()], value=None, placeholder="Todos")], width=3),
        dbc.Col([dcc.DatePickerRange(id='date_picker', start_date=df['date'].min(), end_date=df['date'].max())], width=6)
    ]),
    dbc.Row([
        dbc.Col(html.Div(id='kpi_total_sales'), width=3),
        dbc.Col(html.Div(id='kpi_avg_ticket'), width=3),
        dbc.Col(html.Div(id='kpi_units'), width=3),
        dbc.Col(html.Div(id='kpi_margin'), width=3)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='ts_sales'), width=8),
        dbc.Col(dcc.Graph(id='top_products'), width=4)
    ]),
    dbc.Row(dbc.Col(dcc.Graph(id='sales_heatmap'), width=12))
])

@app.callback(
    [Output('kpi_total_sales','children'),
     Output('kpi_avg_ticket','children'),
     Output('kpi_units','children'),
     Output('kpi_margin','children'),
     Output('ts_sales','figure'),
     Output('top_products','figure'),
     Output('sales_heatmap','figure')],
    [Input('region_drop','value'), Input('product_drop','value'),
     Input('date_picker','start_date'), Input('date_picker','end_date')]
)
def update(region, product, start, end):
    d = df.copy()
    if region:
        d = d[d['region']==region]
    if product:
        d = d[d['product']==product]
    d = d[(d['date']>=start) & (d['date']<=end)]
    total_sales = d['sales'].sum()
    avg_ticket = (d['sales']/d['transactions']).mean() if 'transactions' in d.columns else d['sales'].mean()
    units = d['units'].sum() if 'units' in d.columns else None
    margin = d['margin'].mean() if 'margin' in d.columns else None

    ts = d.groupby('week')['sales'].sum().reset_index()
    fig_ts = px.line(ts, x='week', y='sales', title='Ventas Semanales')
    top = d.groupby('product')['sales'].sum().nlargest(10).reset_index()
    fig_top = px.bar(top, x='product', y='sales', title='Top 10 Productos')
    heat = d.groupby(['region','store'])['sales'].sum().reset_index()
    fig_heat = px.treemap(heat, path=['region','store'], values='sales', title='Ventas por región/tienda')

    return (f"${total_sales:,.0f}", f"${avg_ticket:,.2f}", f"{units:,}" if units is not None else "N/A",
            f"{margin:.2%}" if margin is not None else "N/A", fig_ts, fig_top, fig_heat)

if __name__ == "__main__":
    app.run(debug=True)


---
**Actividades adicional**

1. Añadir un control para comparar periodo actual vs mismo periodo del año anterior (YoY).
2. Implementar un *drilldown*: hacer clic en un producto en `top_products` y mostrar detalle por tienda.
3. Crear una vista para analizar *efecto de promoción* (marcar períodos promo y comparar).
---


## Ejercicio 3 — Dashboard macro-financiero (caso: CFO vs macro)

**Contexto:** El CFO quiere ver cómo las ventas reales evolucionan con respecto a la inflación y cómo los retornos del mercado se relacionan con indicadores macro.

**Datasets:**

* **CPI** (Consumer Price Index) desde FRED: puedes descargar CSV directamente en la página de la serie (botón Download → CSV). ([fredhelp.stlouisfed.org][3], [fred.stlouisfed.org][5])
* Índice S\&P 500 (ticker `^GSPC`) o usar `yfinance` para descargar historic. ([macroption.com][1])

**Actividades:**

1. Descargar CPI (mensual) y convertir a índice base (por ejemplo 2010=100).
2. Ajustar ventas por inflación (ventas\_reales = ventas\_nominales / (CPI / CPI\_base)).
3. Mostrar gráfico comparativo: ventas nominales vs ventas reales (ajustadas).
4. Calcular correlación entre retornos del mercado y crecimiento en ventas trimestrales (mostrar heatmap de correlaciones).

**Ejercicio:** construir un dashboard en Dash que permita seleccionar:

* indicador CPI (varias series),
* rango de fechas,
* producto o región,
  y que muestre la versión "real" de la serie de ventas junto con la tasa de inflación YoY.

---


# 3. ACTIVIDADES COMPLEMENTARIAS

1. **Extensión técnica (avanzado):** integrar un cálculo de forecasting (ARIMA o Prophet) en el dashboard y mostrar predicción con intervalos de confianza. Evaluar supuestos y comunicar limitaciones.
2. **Test & QA:** crear tests unitarios para las funciones de cálculo de indicadores (ej. una prueba para drawdown).
3. **Despliegue:** dockerizar la app y desplegar en un servicio (documentar pasos).

---



# 4. RECURSOS / LINKS ÚTILES

* **Yahoo Finance** (descarga histórico o usar `yfinance`). Guía para descargar histórico. ([macroption.com][1])
* **Alpha Vantage** (API gratuita con límite): documentación y sign-up para API key. ([alphavantage.co][2])
* **FRED** (Federal Reserve Economic Data): descargar series macro (CSV/Excel). ([fredhelp.stlouisfed.org][3], [fred.stlouisfed.org][5])
* **Kaggle** (retail datasets): buscar `retail sales` y descargar datasets preparados. ([kaggle.com][4])

---
[1]: https://www.macroption.com/yahoo-finance-download-historical-data/?utm_source=chatgpt.com "How to Download Historical Data from Yahoo Finance - Macroption"
[2]: https://www.alphavantage.co/?utm_source=chatgpt.com "Alpha Vantage: Free Stock APIs in JSON & Excel"
[3]: https://fredhelp.stlouisfed.org/fred/data/downloading/using-the-download-data-link/?utm_source=chatgpt.com "Downloading Data from FRED | Getting To Know FRED"
[4]: https://www.kaggle.com/datasets?fileType=csv&search=retail&utm_source=chatgpt.com "Find Open Datasets and Machine Learning Projects - Kaggle"
[5]: https://fred.stlouisfed.org/series/CPIAUCSL?utm_source=chatgpt.com "Consumer Price Index for All Urban Consumers: All Items in U.S. ..."
